In [43]:
import sys
import os
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)
import system.robot as robotSystem
import casadi as ca
import system.utils.transformation_matrix as Transformation
import numpy as np

In [44]:
alpha = robotSystem.RobotDynamics()
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
path_to_urdf = os.path.join(
    project_root,
    'usage',
    'urdf','reach_alpha_5',
    'alpha_5_robot.urdf'
)
alpha.from_file(path_to_urdf)
root = "base_link"
tip = "alpha_standard_jaws_base_link"
kinematic_dict, K, P, L, D, C, g, B, qdd, joint_torque, sys_id_coeff, F_payload = alpha.build_model(root, tip, floating_base=True)
kinematic_dict = alpha.get_kinematic_dict()
c_parms, m_params, I_params, fv_coeff, fc_coeff, fs_coeff, v_s_coeff, vec_g, r_com_body, m_p, q, q_dot, q_dotdot, tau, base_pose, world_pose, tip_offset_pose = kinematic_dict['parameters']
n_joints = alpha.get_n_joints(root, tip)

In [45]:
# decision variables for IK, these are spatial velocities, not Euler rates
twist_base_W = ca.SX.sym('twist_base_W', 6, 1)  
v_base_W     = twist_base_W[0:3] # base linear velocity in world
omega_base_W = twist_base_W[3:6] # base angular velocity in world

q_dot_cmd = ca.SX.sym('q_dot_cmd', n_joints, 1)
nu = ca.vertcat(twist_base_W, q_dot_cmd)

# position task
Jp_ee_W = kinematic_dict['geo_J_full'][-1][0:3, :]    # linear Jacobian (world)
p_ee_W  = kinematic_dict['Fks'][-1][0:3]              # EE position in world

kp = ca.SX.sym('kp', 3, 1)
p_des = ca.SX.sym('p_des', 3, 1)

# desired end-effector linear velocity (per-axis gains)
v_des = ca.diag(kp) @ (p_des - p_ee_W)                # 3 x 1

# task residual: match task-space linear velocity
e_p_task = Jp_ee_W @ nu - v_des                       # 3 x 1

In [46]:
# tool axis in tool frame (choose one)
a_tool = ca.DM([0, 0, 1])     # z-axis of end effector frame
# a_tool = ca.DM([1, 0, 0])   # x-axis alternative
# a_tool = ca.DM([0, 1, 0])   # y-axis alternative

# desired axis in world frame (unit vector)
a_des = ca.SX.sym('a_des', 3, 1)                 # keep symbolic for function input
# a_des_u = a_des / ca.norm_2(a_des)   # normalized copy, use this in computations
a_des_u = a_des / (ca.norm_2(a_des) + 1e-12)

R_ee_W = kinematic_dict['R_symx'][-1]   # EE rotation in world
a_now = R_ee_W @ a_tool                                   # 3x1, tool axis expressed in world

e_axis = ca.cross(a_now, a_des_u)        # 3x1, zero when aligned
k_axis = ca.SX.sym('k_axis')           # gain (rad/s per unit error)
w_des_axis = k_axis * e_axis          # 3x1 desired angular velocity in world

Jw_ee_W = kinematic_dict['geo_J_full'][-1][3:6, :]   # angular Jacobian (world)                            # 3x1
e_axis_task = Jw_ee_W @ nu - w_des_axis               # 3x1

In [47]:
# roll,pitch soft stabilization (servo Euler angles back toward 0)
phi   = world_pose[3]
theta = world_pose[4]
psi = world_pose[5]
k_rp = ca.SX.sym('k_rp')                         # scalar

# desired Euler rates (stabilize roll, pitch, leave yaw free)
phi_dot_des   = k_rp * (0 - phi)
theta_dot_des = k_rp * (0 - theta)
psi_dot_des   = 0

# your tested mapping: omega_world = Ts(phi,theta,psi) @ euler_dot
Ts = Transformation.euler_to_spatial_rate_T(phi, theta, psi)
euler_dot_des = ca.vertcat(phi_dot_des, theta_dot_des, psi_dot_des)
omega_des_rp  = Ts @ euler_dot_des                        # 3x1, desired world angular velocity

# residual in angular velocity space (consistent with geo Jw)
e_omega = omega_base_W - omega_des_rp   # 3x1

# only penalize roll and pitch behavior, do not fight yaw
e_rp    = e_omega[0:2]                  # 2x1

In [48]:
# relative orientation task between the floating base and the arm’s first link to keep it centered
a_x = ca.DM([1, 0, 0])                      # local x axis

R_B_W  = Transformation.rot_from_rpy(phi, theta, psi)     # base rotation in world
R_J0_W = kinematic_dict['R_symx'][0]                      # joint0 rotation in world

x_B_W  = R_B_W  @ a_x                      # base x axis in world
x_J0_W = R_J0_W @ a_x                     # joint0 x axis in world

Jw_J0_W    = kinematic_dict['geo_J_full'][0][3:6, :]  # joint0 angular Jacobian
omega_J0_W = Jw_J0_W @ nu                 # joint0 angular velocity in world

omega_rel = omega_J0_W - omega_base_W    # joint0 relative to base

e_align = ca.cross(x_J0_W, -x_B_W)        # axis misalignment error
k_align = ca.SX.sym('k_align')
w_des_align = k_align * e_align           # desired relative angular velocity

e_align_task = omega_rel - w_des_align    # alignment residual

In [49]:
# regularization (keeps solution small, avoids base thrashing)
e_reg = nu                                       # (6+n) x 1

In [50]:
# weighted least squares objective
w_rp  = ca.SX.sym('w_rp')                        # scalar
w_reg = ca.SX.sym('w_reg')                       # scalar
w_axis = ca.SX.sym('w_axis')  # scalar
w_align = ca.SX.sym('w_align')
cost = (
    ca.dot(e_p_task, e_p_task)
    + w_axis * ca.dot(e_axis_task, e_axis_task)
    + w_rp  * ca.dot(e_rp, e_rp)
    + w_reg * ca.dot(e_reg, e_reg)
    + w_align * ca.dot(e_align_task, e_align_task)
    
)

In [51]:
# 1. Compute Hessian and Full Gradient
H, g_full = ca.hessian(cost, nu)

# 2. Extract the linear term of the gradient
# The full gradient is (H * nu + g_linear). 
# By substituting nu=0, we isolate g_linear.
g_linear = ca.substitute(g_full, nu, ca.DM.zeros(nu.shape))

# 3. Solve for the optimal nu
# Analytic solution for QP: H * nu + g_linear = 0  =>  nu = -inv(H) * g_linear
lam = 1e-8 + 1e-6 * w_reg
nu_star = -ca.solve(H + lam * ca.SX.eye(H.size1()), g_linear)

In [52]:
dt = ca.SX.sym('dt')

# linear part integrates directly
p_world_next = world_pose[0:3] + nu_star[0:3] * dt

# convert omega -> euler_dot for integration
Ts = Transformation.euler_to_spatial_rate_T(phi, theta, psi)

euler_dot_star = ca.solve(Ts, nu_star[3:6])               # 3x1

rpy_next = world_pose[3:6] + euler_dot_star * dt



x_world_ref_next = ca.vertcat(p_world_next, rpy_next)
q_ref_next       = q + nu_star[6:6+n_joints] * dt

# Pack state = [world_pose; q]
state0 = ca.vertcat(world_pose, q)  # (6+n) x 1

# One IK step as a CasADi function (state -> next_state)
state1 = ca.vertcat(x_world_ref_next, q_ref_next)  # (6+n) x 1

e_p_task_star = Jp_ee_W @ nu_star - v_des
e_axis_task_star = Jw_ee_W @ nu_star - w_des_axis

ik_step = ca.Function(
    'ik_step',
    [state0, kp, p_des, w_rp, w_reg, k_rp, a_des, k_axis, w_axis, w_align, k_align, dt, base_pose, tip_offset_pose],
    [state1, e_p_task_star, e_axis_task_star]
)

In [53]:
# Mapaccum over N iterations
N = 5000
ik_step_acc = ik_step.mapaccum('ik_step_acc', N)

# Accumulated trajectories
state_traj, e_p_traj, e_axis_traj = ik_step_acc(
    state0, kp, p_des, w_rp, w_reg, k_rp, a_des, k_axis, w_axis, w_align, k_align, dt, base_pose, tip_offset_pose
)

In [54]:
# Take the final values (last column)
stateN = state_traj[:, -1]
x_world_next = stateN[0:6]
q_next       = stateN[6:6+n_joints]

e_p_task_star_new    = e_p_traj[:, -1]
e_axis_task_star_new = e_axis_traj[:, -1]

ik_func = ca.Function(
    'mapacc_task_based_ik',
    [q, world_pose, kp, p_des, w_rp, w_reg, k_rp, a_des, k_axis, w_axis, w_align, k_align, dt, base_pose, tip_offset_pose],
    [x_world_next, q_next, e_p_task_star_new, e_axis_task_star_new]
)

In [55]:
q_val=[1.0002, 1.9335, 0.3438, 0.0125]
world_pose_val=[ 4.1448e-01 ,-1.5551e-01, -1.3213e+00,  3.7661e-05,  3.1087e-04 , 9.7403e-01]
p_des_val=[ 5.5038e-01 ,-1.2290e-17 ,-1.5958e+00]
a_des_val=[-1. , 0. , 0.]
tool_axis_val=[-1.,  0.,  0.]

kp_val = np.array([1.0, 1.0, 1.0], dtype=float)
w_rp_val = 1.0
w_reg_val = 0.02
k_rp_val = 0.2
k_axis_val = 1.0
w_axis_val = 1.5
dt_val = 1.0 / 500.0
w_align_val = 1.0
k_align_val = 1.0

base_pose_val = np.asarray([0.190, 0.000, -0.120, 3.141592653589793, 0.000, 0.000], dtype=float)
tip_offset_pose_val = np.asarray([0.00, 0.00, 0.04, 0.00, 0.00, 0.00], dtype=float)

ik_func(q_val, world_pose_val, kp_val, p_des_val, w_rp_val, w_reg_val, k_rp_val, a_des_val, k_axis_val, w_axis_val, w_align_val, k_align_val, dt_val, base_pose_val, tip_offset_pose_val)

(DM([0.39215, -0.000115922, -1.3195, -0.00255533, 0.00277426, 0.00480391]),
 DM([3.14195, 1.93262, 0.362194, -0.939636]),
 DM([5.27405e-06, -1.28815e-05, -3.15129e-06]),
 DM([6.75546e-06, 1.28602e-05, 6.34249e-05]))

In [56]:
ik_func.save('ik_whole_b.casadi')